In [1]:
cd ..

/Users/kangjiwon/Desktop/LG Aimers


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
# %rm -rf phase2_module
# !git clone https://github.com/LgDNet/phase2_module.git
# !cd phase2_module && make copy

In [3]:
from phase2_module.src.utils.dataset import Data
from phase2_module.src.models.classification.models import Model
from phase2_module.src.base import BasePiepline
# from phase2_module.src.hyper_parameters.params_optimization import xgboost_opt, set_params_optimization_data
from phase2_module.src.utils.set_seed import seed_everything
from phase2_module.src.utils.top_score_instance import check_the_score

from phase2_module.src.utils.manage_pkl_files import PickleManager as pkl_manager
from phase2_module.src import preprocess
seed_everything()  # NOTE: default 42
_model = Model()

In [4]:
import pandas as pd
import numpy as np
import re, pickle
import warnings
from tqdm import tqdm
warnings.filterwarnings(action='ignore')
def set_up(mode= None):
    pd.set_option('display.max_columns', mode)
set_up()

In [5]:
from sklearn.feature_selection import RFECV 
from sklearn.tree import DecisionTreeClassifier 

# Preprocessing

In [6]:
basic = preprocess.Basic
country = preprocess.Country
customer = preprocess.Customer
inquiry = preprocess.Inquiry
product_category = preprocess.ProductCategory
convert_ratio = preprocess.Convert_ratio

# Main

In [7]:
pkls = pkl_manager.map(pkl_manager.loads, pkl_manager.metadata_directory)

runtime_instance = {
    "Train": Data.train,
    "Test": Data.test,
    "Instances":[
        {"instance": basic, "method": ['drop_duplicated'], "params": False},
        {"instance": country, "method": ['apply'], "params": pkls.get("country")},
        {"instance": inquiry, "method": ["apply"], "params": False},
        {"instance": product_category, "method": ["apply"], "params": {**pkls.get("product_category"), **pkls.get("product_subcategory")}},
        {"instance": customer, "method": ["apply"], "params": pkls.get("customer")},
        {"instance": convert_ratio, "method": ["lead_owner","customer_idx",'lead_owner_customer_idx_means'], "params": {**pkls.get("lead_owner"), **pkls.get("customer_idx")}}
    ]
}

## preprocessing

In [14]:
df_train = preprocess.runner('Train',**runtime_instance)
df_test = preprocess.runner('Test',**runtime_instance)

## encoding

In [15]:
from sklearn.preprocessing import LabelEncoder
class Encode:
    def __init__(self):
        pass
    def label_encoder(self,df,columns):
        if not encoding_zip['label']:
            for col in columns: # train
                encoder = LabelEncoder()
                try:
                    df.loc[:,col] = encoder.fit_transform(df[col])
                except Exception as e:
                    print(e)
                    print(col)
                # 인코더 저장
                encoding_zip['label'].append(encoder)
        else: # test
            for idx, col in enumerate(columns):
                encoder = encoding_zip['label'][idx]
                
                for label in np.sort(df[col].unique()):
                    if label not in encoder.classes_:
                        encoder.classes_ = np.append(encoder.classes_,label)
                df.loc[:,col] = encoder.transform(df[col])
            
        return df

In [16]:
encode = Encode()
encoding_zip = {"label":[], "onehot":[]}

In [17]:
exisiting_columns = [
                     'business_area','business_subarea','business_unit',
                        'enterprise',
#                         'response_corporate',"customer_country", # country
#                         'customer_type','customer_job','customer_position', #customer
#                         'inquiry_type','expected_timeline', # inquiry
#                         'product_subcategory','product_category','product_modelname' # product_category
                       ] 
preprocess_columns = [
                     'country','continent', # country
                     'customer_type2','job_function','seniority_level', # customer
                     'new_inquiry_type','new_expected_timeline', # inquiry
                     'category_1','category_2','category_3' # product_category
                     ]

In [18]:
label_encode_columns = exisiting_columns + preprocess_columns

In [19]:
# 결측치 채우기
def fillna(df):
    for col in df.columns:
        if df[col].isna().sum():
            if df[col].dtypes =='O':
                df[col].fillna('ETC', inplace = True)
            else:
                df[col].fillna(0,inplace = True)
    return df
df_train = fillna(df_train)
df_test = fillna(df_test)

In [20]:
df_trian = encode.label_encoder(df_train,label_encode_columns)
df_test = encode.label_encoder(df_test,label_encode_columns)

In [46]:
X = df_train.drop(["is_converted"],axis = 1)
Y = df_train['is_converted']

In [67]:
  # Create a decision tree classifier 
dict_ = {'criterion': 'log_loss', 'max_depth': 11, 'min_samples_split': 7, 'min_samples_leaf': 2, 'random_state': 42}
estimator = DecisionTreeClassifier() 
estimator.set_params(**dict_)
  
# Use RFE with cross-validation to  
# find the optimal number of features 
selector = RFECV(estimator, step=1, cv=5, min_features_to_select=20)
selector = selector.fit(X, Y) 
  
# Print the optimal number of features 
print("Optimal number of features: %d" % selector.n_features_) 
  
# Print the selected features 
print("Selected features: %s" % selector.support_) 

Optimal number of features: 20
Selected features: [ True  True  True  True  True  True False False False  True  True  True
  True  True False  True  True  True False  True  True  True  True  True
 False  True]


In [68]:
feature_importances = selector.support_

In [69]:
feature_names = X.columns

In [70]:
feature_importance_dict = dict(zip(feature_names, feature_importances))

sorted_importance_dict = {k: v for k, v in sorted(feature_importance_dict.items(), key=lambda item: item[1], reverse=True)}
for feature_name, importance in sorted_importance_dict.items():
    print(f"{feature_name}: {importance}")

bant_submit: True
business_unit: True
com_reg_ver_win_rate: True
enterprise: True
historical_existing_cnt: True
lead_desc_length: True
business_area: True
business_subarea: True
lead_owner: True
country: True
continent: True
new_expected_timeline: True
customer_interest: True
category_1: True
category_3: True
customer_type2: True
job_function: True
seniority_level: True
lead_owner_converted_ratio: True
customer_idx_group: True
ver_cus: False
ver_win_rate_x: False
ver_win_ratio_per_bu: False
new_inquiry_type: False
category_2: False
lead_owner_customer_idx_mean: False


# Modeling

## drop column
모델링 전이 column을 drop하면서 다루기가 쉽다고 판단하였음

In [40]:
pwd

'/Users/kangjiwon/Desktop/LG Aimers'

In [44]:
df_train = pd.read_csv('phase2_module/data/best_score_train.csv')

In [45]:
drop_col = [
            'id_strategic_ver','it_strategic_ver',
            'customer_country.1',
    
            'response_corporate',"customer_country", # country
            'customer_type','customer_job','customer_position', # customer
            'inquiry_type', 'expected_timeline', #inquiry
            'product_subcategory','product_category','product_modelname', # product_category
#             'cate_is_nan', # product_category
    
#             'com_reg_ver_win_rate','idit_strategic_ver','ver_cus',
#             'ver_pro','ver_win_rate_x','ver_win_ratio_per_bu',
            'customer_idx',
            'customer_idx_converted_ratio', # ratio
            'idit_strategic_ver','ver_pro' # test
            ]
df_train.drop(drop_col,axis = 1, inplace = True)

# df_test.drop(drop_col,axis = 1, inplace = True)

KeyError: "['id_strategic_ver', 'it_strategic_ver', 'customer_country.1', 'response_corporate', 'customer_country', 'customer_type', 'customer_job', 'customer_position', 'inquiry_type', 'expected_timeline', 'product_subcategory', 'product_category', 'product_modelname', 'customer_idx', 'customer_idx_converted_ratio', 'idit_strategic_ver', 'ver_pro'] not found in axis"

## Scaler

In [290]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df_train[['lead_owner_converted_ratio','lead_owner_customer_idx_mean']] = scaler.fit_transform(df_train[['lead_owner_converted_ratio','lead_owner_customer_idx_mean']])
df_test[['lead_owner_converted_ratio','lead_owner_customer_idx_mean']] = scaler.transform(df_test[['lead_owner_converted_ratio','lead_owner_customer_idx_mean']])
# df_test['lead_desc_length'] = scale.transform(df_test['lead_desc_length'].values.reshape(-1, 1))

## train

In [335]:
model = _model.decision_tree

In [336]:
from sklearn.ensemble import RandomForestClassifier

In [337]:
model.set_params(random_state=42)#, min_samples_leaf=5, max_depth = 10)

DecisionTreeClassifier(random_state=42)

In [338]:
X = df_train.drop(["is_converted"],axis = 1)
Y = df_train['is_converted']

In [339]:
from sklearn.metrics import roc_auc_score, confusion_matrix, auc
from sklearn.model_selection import StratifiedKFold
stratkfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
result = {"f1":[], "precision":[], "recall":[],'roc_auc_score':[]}
train_result = {"f1":[], "precision":[], "recall":[],'roc_auc_score':[]}

# k-fold
for train_idx, test_idx in tqdm(stratkfold.split(X,Y)):
    x_train, x_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = Y[train_idx], Y[test_idx]

    # 모델 훈련
    model.fit(x_train, y_train)
    
    train_pred = model.predict(x_train)
    predict = model.predict(x_test)
    
    score_result = check_the_score(train_pred, y_train)
    for name, score in score_result.items():
        train_result[name].append(score)
        
    score_result = check_the_score(predict, y_test)
    
#     for i,j in zip(predict, y_test):
        
    for name, score in score_result.items():
        result[name].append(score)
        
print('----[K-Fold Train Score]-----')
for name, score_list in train_result.items():
    print(f'{name} score : {np.mean(score_list):.4f} / STD: (+/- {np.std(score_list):.4f})')
    
print('----[K-Fold Validation Score]-----')
for name, score_list in result.items():
    print(f'{name} score : {np.mean(score_list):.4f} / STD: (+/- {np.std(score_list):.4f})')

5it [00:01,  2.91it/s]

----[K-Fold Train Score]-----
f1 score : 0.9843 / STD: (+/- 0.0003)
precision score : 0.9736 / STD: (+/- 0.0019)
recall score : 0.9951 / STD: (+/- 0.0014)
roc_auc_score score : 0.9964 / STD: (+/- 0.0006)
----[K-Fold Validation Score]-----
f1 score : 0.6417 / STD: (+/- 0.0101)
precision score : 0.6377 / STD: (+/- 0.0153)
recall score : 0.6462 / STD: (+/- 0.0136)
roc_auc_score score : 0.8068 / STD: (+/- 0.0069)


In [317]:
feature_importances = model.feature_importances_
feature_names = x_train.columns
feature_importance_dict = dict(zip(feature_names, feature_importances))
sorted_importance_dict = {k: v for k, v in sorted(feature_importance_dict.items(), key=lambda item: item[1], reverse=True)}
for feature_name, importance in sorted_importance_dict.items():
    print(f"{feature_name}: {importance}")

lead_owner_converted_ratio: 0.3766756623601492
customer_idx: 0.3752871553806524
lead_desc_length: 0.03913853320652947
customer_type2: 0.025983912440296412
lead_owner: 0.025193189381263564
job_function: 0.018949077375517608
business_subarea: 0.018780395458546907
seniority_level: 0.013583074041902538
category_3: 0.012487755802031283
country: 0.011026955910926567
historical_existing_cnt: 0.01065736904861266
bant_submit: 0.00906398477750914
com_reg_ver_win_rate: 0.008199327244543753
new_expected_timeline: 0.008167698380358052
continent: 0.007067369888900143
lead_owner_customer_idx_mean: 0.006928559044672509
category_2: 0.006562150335444641
enterprise: 0.0059110422724759845
ver_win_rate_x: 0.003772253623530848
business_area: 0.003318642346996751
business_unit: 0.002943025921724158
customer_interest: 0.0025773413458897764
ver_cus: 0.002121992204650687
ver_win_ratio_per_bu: 0.0016928120855966358
new_inquiry_type: 0.0014531013965146322
category_1: 0.0012723506227941043
idit_strategic_ver: 0.00

## test & submission

In [340]:
test_drop = df_test.drop(['id','is_converted'],axis = 1)

In [341]:
pred = model.predict(test_drop)
pred = np.where(pred == 0, False, True)

In [342]:
sum(pred)

881

In [344]:
su = 0
for i,j in zip(pred1,pred2):
    if i != j:
        su +=1
su

635

In [ ]:
# customer_idx 유무 차이
- > 635

In [148]:
df_test['is_converted'] = pred
df_test.to_csv("submission.csv",index = False)

In [108]:
# df_train.to_csv("preprocessing_derivation_train.csv",index = False)

In [109]:
# df_test.to_csv("preprocessing_derivation_test.csv",index = False)